In [1]:
import numpy as np
import os
import pickle
from IPython.display import clear_output
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.rcParams['figure.figsize'] = [15, 5]
import sys
sys.path.insert(1, os.getcwd()+"/../..") # Not needed if you installed SyMBac using pip
from SyMBac.colony_simulation import ColonySimulation
from SyMBac.colony_renderer import ColonyRenderer
from SyMBac.PSF import PSF_generator
import numpy as np
from scipy import fftpack
from scipy.ndimage import gaussian_filter
from utils import GetSourcePts, GetMidPts, InverseMatrix, ApproxPSFBesselOptimise, GetAvgIntensityWithMask
import cv2

#Remove directory data if it exists
if os.path.exists("data"):
    os.system("rm -rf data")

/home/boomalope/anaconda3/envs/symbac/SyMBac/tests/dx_test_env/../../SyMBac/colony_simulation.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
TRIALS = 10
PATH = os.getcwd() 
SAVED_FOLDER = "/test"
DATA = []
NA_USED = [1, 2, 5, 10, 20]
ASSUMED_NOISE_LEVEL = 1e7

for t in range(TRIALS):

    #Create directory data if it does not exist
    NEW_FOLDER = PATH+SAVED_FOLDER+"/"+str(t)
    if not os.path.exists(NEW_FOLDER):
        os.makedirs(NEW_FOLDER)

    print("Current trial: ", t)
    my_kernel = PSF_generator(
        radius = 100, 
        wavelength = 0.75, 
        NA = 1.45, 
        n = 1.4, 
        resize_amount = 3,
        pix_mic_conv = 0.065, 
        apo_sigma = 8, 
        mode="simple fluo", 
        condenser = "Ph3",
        offset = 0.02
    )
    my_kernel.calculate_PSF()

    while True:
        try:
            colonysim = ColonySimulation(
                cellmodeller_model= 'cellmodeller_ex1_simpleGrowth_modified.py',
                max_cells = 7,
                pix_mic_conv = 0.065,
                resize_amount = 3,
                save_dir = "dx_test/",
            )

            colonysim.run_cellmodeller_sim(num_sim=1)
            colonysim.get_simulation_dirs()
            pickles = colonysim.get_simulation_pickles()
            colonysim.get_max_scene_size()

            FRAME_CNT = len(pickles[-1])

            colonysim.draw_simulation_OPL(n_jobs = 1, FL=True, density = 0.1, random_distribution = "uniform", distribution_args = (0.9, 3))
            break
        except:
            #Remove directory data if it exists
            if os.path.exists("data"):
                os.system("rm -rf data")
            clear_output(wait=True)
            pass

    my_renderer = ColonyRenderer(colonysim, my_kernel)
    out = my_renderer.render_scene(-1)
    psf = my_kernel.get_fluorescence_kernel(my_kernel.wavelength,my_kernel.NA,my_kernel.n,my_kernel.radius,my_kernel.scale)
    ori = mpimg.imread(PATH + '/data/scenes/00{frame}.png'.format(frame=FRAME_CNT))
    mask = mpimg.imread(PATH + '/data/masks/00{frame}.png'.format(frame=FRAME_CNT))    

    with open(NEW_FOLDER+'/ori.pkl', 'wb') as file: 
        pickle.dump(ori, file)
    with open(NEW_FOLDER+'/mask.pkl', 'wb') as file: 
        pickle.dump(mask, file)
    with open(NEW_FOLDER+'/out.pkl', 'wb') as file: 
        pickle.dump(out, file)
    with open(NEW_FOLDER+'/psf.pkl', 'wb') as file: 
        pickle.dump(psf, file)

    #Run pipeline

    ori = ori/max(ori.flatten())
    mask = mask/max(mask.flatten())
    out = out/max(out.flatten())

    #Apply gaussian filter to out
    outBlur = gaussian_filter(out, sigma=1)

    #Resize mask to match out
    maskRescale = cv2.resize(mask, outBlur.shape[::-1], interpolation = cv2.INTER_NEAREST)

    #IMPT: All mask values are set to one
    maskUnit = maskRescale.copy()

    for i in range(len(maskUnit)):
        for j in range(len(maskUnit[0])):
            if (maskUnit[i][j] != 0):
                maskUnit[i][j] = 1
    
    # Compute the 2d FFT of the input image
    oriFreq = fftpack.fft2(ori)
    maskFreq = fftpack.fft2(maskUnit)
    outFreq = fftpack.fft2(outBlur)

    # Shift the zero-frequency component to the center of the spectrum.
    oriFreq = fftpack.fftshift(oriFreq)
    maskFreq = fftpack.fftshift(maskFreq)
    outFreq = fftpack.fftshift(outFreq)

    #Transform mask.shape into odd values

    psfShape = [0,0]
    psfShape[0] = mask.shape[0] + 1 if mask.shape[0] % 2 == 0 else mask.shape[0]
    psfShape[1] = mask.shape[1] + 1 if mask.shape[1] % 2 == 0 else mask.shape[1]

    psfFreqMask= outFreq * np.conj(maskFreq) / (np.abs(maskFreq)**2 + ASSUMED_NOISE_LEVEL/np.max(np.abs(maskFreq)**2))
    psfMask = fftpack.ifft2(psfFreqMask, shape=psfShape)
    psfMask = fftpack.fftshift(psfMask)
    psfModel = ApproxPSFBesselOptimise(np.abs(psfMask))

    sourcePts = GetSourcePts(mask)
    sourcePtsRescale = GetSourcePts(maskRescale)
    recoveredImageAllPixels = InverseMatrix(outBlur, maskRescale, sourcePtsRescale, adjPts=0, learningRate = 1, psf=psfModel)
    midPts = GetMidPts(maskRescale)
    recoveredImageMidPts = InverseMatrix(outBlur, maskRescale, midPts, adjPts=0, learningRate = 1, psf=psfModel)

    average_original_mask, average_original_intensity = GetAvgIntensityWithMask(ori,mask,sourcePts)
    average_avg_output_mask, average_avg_output_intensity = GetAvgIntensityWithMask(out,maskRescale,sourcePtsRescale)
    average_all_pixels_mask, average_all_pixels_intensity = GetAvgIntensityWithMask(recoveredImageAllPixels,maskRescale,sourcePtsRescale)
    average_mid_pts_mask, average_mid_pts_intensity = GetAvgIntensityWithMask(recoveredImageMidPts,maskRescale,midPts)

    # Sort average_original_mask into ascending order and sort average_original_intensity accordingly
    average_original_mask, average_original_intensity = zip(*sorted(zip(average_original_mask, average_original_intensity)))
    average_avg_output_mask, average_avg_output_intensity = zip(*sorted(zip(average_avg_output_mask, average_avg_output_intensity)))
    average_all_pixels_mask, average_all_pixels_intensity = zip(*sorted(zip(average_all_pixels_mask, average_all_pixels_intensity)))
    average_mid_pts_mask, average_mid_pts_intensity = zip(*sorted(zip(average_mid_pts_mask, average_mid_pts_intensity)))

    for i in range(len(average_original_mask)):
        assert(average_original_mask[i] == average_avg_output_mask[i])
        assert(average_original_mask[i] == average_all_pixels_mask[i])
        assert(average_original_mask[i] == average_mid_pts_mask[i])

    correlation_avg_output = np.corrcoef(average_original_intensity,average_avg_output_intensity)[0][1]
    correlation_all_pixels = np.corrcoef(average_original_intensity,average_all_pixels_intensity)[0][1]
    correlation_mid_pts = np.corrcoef(average_original_intensity,average_mid_pts_intensity)[0][1]

    print("Simple Averaging: ", correlation_avg_output)
    print("All Pixels: ", correlation_all_pixels)
    print("Mid Pts: ", correlation_mid_pts)

    all_impt_info = []
    all_impt_info.append(correlation_avg_output)
    all_impt_info.append(correlation_all_pixels)
    all_impt_info.append(correlation_mid_pts)

    all_impt_info.append(my_kernel.radius)
    all_impt_info.append(my_kernel.wavelength)
    all_impt_info.append(my_kernel.NA)
    all_impt_info.append(my_kernel.n)
    all_impt_info.append(my_kernel.resize_amount)
    all_impt_info.append(my_kernel.pix_mic_conv)
    all_impt_info.append(my_kernel.apo_sigma)
    all_impt_info.append(my_kernel.mode)
    all_impt_info.append(my_kernel.condenser)
    all_impt_info.append(my_kernel.offset)

    all_impt_info.append(colonysim.cellmodeller_model)
    all_impt_info.append(colonysim.max_cells)
    all_impt_info.append(colonysim.pix_mic_conv)
    all_impt_info.append(colonysim.resize_amount)    

    DATA.append(all_impt_info)

    #Remove directory data if it exists
    if os.path.exists("data"):
        os.system("rm -rf data")
    
    #Clear Jupyter output
    clear_output(wait=True)



Current trial:  9
Creating directory data/dx_test/
Set up OpenCL context:
  Platform: NVIDIA CUDA
  Device: NVIDIA GeForce GTX 1650
Importing model cellmodeller_ex1_simpleGrowth_modified
      10           2 cells           0 contacts    0.000018 hour(s) or 0.001087 minute(s) or 0.065250 second(s)
      20           2 cells           0 contacts    0.000030 hour(s) or 0.001773 minute(s) or 0.106359 second(s)
      30           2 cells           0 contacts    0.000043 hour(s) or 0.002566 minute(s) or 0.153940 second(s)
      40           2 cells           0 contacts    0.000054 hour(s) or 0.003232 minute(s) or 0.193898 second(s)
      50           3 cells           1 contacts    0.000074 hour(s) or 0.004436 minute(s) or 0.266150 second(s)
   50     3 cells       1 cts       2 iterations  residual = 0.000008
      60           3 cells           1 contacts    0.000101 hour(s) or 0.006089 minute(s) or 0.365366 second(s)
   60     3 cells       1 cts       2 iterations  residual = 0.000012
 

Scene Draw:: 0it [00:00, ?it/s]

/home/boomalope/anaconda3/envs/symbac/SyMBac/tests/dx_test_env/../../SyMBac/PSF.py:178: RuntimeWarning: invalid value encountered in divide
  PSF = (2 * jv(1, rr) / (rr)) ** 2


[101.15409682] [[0.10028556]]


/home/boomalope/anaconda3/envs/symbac/SyMBac/tests/dx_test_env/utils.py:341: RuntimeWarning: invalid value encountered in scalar divide
  return (2*jv(1,x*a) / (x*a))**2


[0.14285715, 0.2857143, 0.42857143, 0.5714286, 0.71428573, 0.85714287, 1.0]
[[0.87094901]
 [0.46010154]
 [1.        ]
 [0.88065827]
 [0.71693635]
 [0.59973135]
 [0.41972269]]
[0.14285715, 0.2857143, 0.42857143, 0.5714286, 0.71428573, 0.85714287, 1.0]
[[0.75098685]
 [0.6144983 ]
 [1.        ]
 [0.86735063]
 [0.68944056]
 [0.68219065]
 [0.43553822]]
Simple Averaging:  0.8834845429048722
All Pixels:  0.9885051449072907
Mid Pts:  0.9361772856817473


In [4]:
print(DATA)

[[0.948163510104294, 0.997482746801115, 0.964838306587032, 100, 0.75, 1.45, 1.4, 3, 0.065, 8, 'simple fluo', 'Ph3', 0.02, 'cellmodeller_ex1_simpleGrowth_modified.py', 7, 0.065, 3], [0.7637091103951211, 0.9504054922601163, 0.929914418272733, 100, 0.75, 1.45, 1.4, 3, 0.065, 8, 'simple fluo', 'Ph3', 0.02, 'cellmodeller_ex1_simpleGrowth_modified.py', 7, 0.065, 3], [0.9741227207770096, 0.9975399210778118, 0.9893318776078126, 100, 0.75, 1.45, 1.4, 3, 0.065, 8, 'simple fluo', 'Ph3', 0.02, 'cellmodeller_ex1_simpleGrowth_modified.py', 7, 0.065, 3], [0.5863310308057523, 0.9465116596334778, 0.8133957428856058, 100, 0.75, 1.45, 1.4, 3, 0.065, 8, 'simple fluo', 'Ph3', 0.02, 'cellmodeller_ex1_simpleGrowth_modified.py', 7, 0.065, 3], [0.3771363612535733, 0.4874137077726119, 0.2952606928213984, 100, 0.75, 1.45, 1.4, 3, 0.065, 8, 'simple fluo', 'Ph3', 0.02, 'cellmodeller_ex1_simpleGrowth_modified.py', 7, 0.065, 3], [0.8775242962380626, 0.9648903735502246, 0.8414681050043, 100, 0.75, 1.45, 1.4, 3, 0.065